### Create Technical Inclusion Areas, A raster based approach.
In this notebook, we create technical inclusion areas for utility scale solar or wind installations. The corresponding environmental exclusion layer and technical threshold raster layers will later be masked from this layer in order to generate the final SL1 Inclusion Area layers and subsequent area assessments. 

Make sure all inputs in the selected input geodatabase have the same projection as the template raster *prior* to running this script. You will need to run this notebook separately for wind and solar inputs to generate their respective 'Technical Inclusion' layers. 

In [1]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

#### Set input and output paths, set environmental variables for raster analyses

In [13]:
# Set input and output folder
mainInputFolder = "C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data"  # enter path to input folder
mainOutputFolder = "C:\\Users\\Zachary\\ASSET\\supplyCurve\\analysis\\data\\"  # enter path to output folder

# Set path to input file geodatabase
input_gdb = os.path.join(mainInputFolder, "technoExclusionWind.gdb")
 
# Create folder for results
results_folder = os.path.join(mainOutputFolder, "technicalInclusionAreas")

# Sets environment workspace to input_gdb
arcpy.env.workspace = input_gdb

# Set path to scratch folder
scratch_folder = os.path.join(mainOutputFolder, "scratch")

if not os.path.exists(scratch_folder):
    os.makedirs(scratch_folder)

# Set path to remap tables
remap_table_gdb = os.path.join(mainInputFolder, "remapWind.gdb") # enter path to remap table geodatabase

In [3]:
# Set path to template raster
templateRaster = os.path.join(mainInputFolder, "technoExclusionWindRasters\\SRTM_250m_proj_cl.tif") ##^^ enter path to DEM data

In [4]:
# Set environments for raster analyses
arcpy.env.snapRaster = templateRaster
arcpy.env.extent = templateRaster
arcpy.env.mask = templateRaster
arcpy.env.cellSize = templateRaster

#### Define useful functions

In [6]:
def euclidean_reclassify(feature_class, remap_table):
    '''
    This function calculates the Euclidean distance from a feature class and reclassifies the resulting distance raster
    using a provided remap table.
    '''

    try:
        # Create the euclidean raster name
        euclidean_raster_name = os.path.splitext(feature_class)[0] + "_euclidean.tif" 

        # Create Euclidean distance raster path 
        euclidean_raster = os.path.join(scratch_folder, euclidean_raster_name)

        # Create the reclassified raster path
        reclassified_raster_name = os.path.splitext(feature_class)[0] + "_reclass.tif"

        # Create the reclassified raster path
        reclassified_raster_path = os.path.join(scratch_folder, reclassified_raster_name)

        # Get the remap table path
        remap_table_path = os.path.join(remap_table_gdb, remap_table)

        # Parse the geodatabase table and create a remap object
        remap_ranges = []
        with arcpy.da.SearchCursor(remap_table_path, ["FromValue", "ToValue", "OutputValue"]) as cursor:
            for row in cursor:
                remap_ranges.append(row)

        # Create a RemapRange object
        remap = arcpy.sa.RemapRange(remap_ranges)

        # Get Euclidean distance raster
        arcpy.gp.EucDistance_sa(feature_class, euclidean_raster, "")
        print("Euclidean distance raster generated.")

        # Reclassify euclidean distance raster
        reclassified_raster = Reclassify(in_raster=euclidean_raster, reclass_field="VALUE", remap=remap, missing_values="DATA")
        reclassified_raster.save(reclassified_raster_path)
        print("Euclidean distance raster reclassified successfully.")

        # Clean up the in-memory Euclidean distance raster
        arcpy.Delete_management(euclidean_raster)
        
    except arcpy.ExecuteError:
        print(arcpy.GetMessages())
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [7]:
def get_feature_classes(workspace):
    '''
    This function grab all feature classes from a folder or geodatabase.
    '''

    # Get the current workspace
    previous_workspace = arcpy.env.workspace

    try:
        # Set the new workspace
        arcpy.env.workspace = workspace

        # Get a list of all feature classes in the workspace
        feature_classes = arcpy.ListFeatureClasses()

        # Return the list of feature classes
        return feature_classes

    finally:
        # Reset the workspace to the previous workspace
        arcpy.env.workspace = previous_workspace

In [8]:

def get_rasters(workspace):
    '''
    This function grabs all the rasters from a given folder or geodatabase.
    '''

    # Get the current workspace
    previous_workspace = arcpy.env.workspace

    try:
        # Set the new workspace
        arcpy.env.workspace = workspace
        
        # define wildcard parameter
        wildCard = "*reclass.tif" # alter according to your naming conventions

        # Get a list of all feature classes in the workspace
        rasters = arcpy.ListRasters(wildCard)

        # Return the list of feature classes
        return rasters

    finally:
        # Reset the workspace to the previous workspace
        arcpy.env.workspace = previous_workspace

In [22]:
def convert_to_one_bit(input_raster, output_raster):
    '''
    This function alters the bit depth of a raster. It is designed for the output of a Reclassify tool. 
    '''
    
    try:
        # Create a raster object from the input path
        input_raster_obj = arcpy.sa.Raster(input_raster)

        # Reclassify the input raster to binary (one-bit) format
        binary_raster = arcpy.sa.Con(input_raster_obj > 0, 1)

        # Set the pixel type of the output raster to 1_BIT
        arcpy.management.CopyRaster(binary_raster, output_raster, pixel_type="1_BIT")

        print(f"Conversion to one-bit raster completed successfully. Result saved to {output_raster}")

        # Clean up workspace
        try: 
            arcpy.Delete_management(binary_raster)
            arcpy.Delete_management(input_raster_obj)
        except Exception as e:
            print(f"An error occurred while cleaning up: {str(e)}")

    except arcpy.ExecuteError:
        print(arcpy.GetMessages())
    except Exception as e:
        print(f"An error occurred: {str(e)}")


#### Generate reclassified euclidean distance rasters

In [ ]:
# Get feature classes for euclidean distance calculation
feature_classes = get_feature_classes(input_gdb)
feature_classes

In [12]:
# Iterate through the feature classes and perform Euclidean distance and reclassification
for feature_class in feature_classes:
    # Specify the remap table corresponding to the feature class
    remap_table = feature_class + "_remap_table"  # Adjust this based on your remap table naming convention

    # Call the function to perform Euclidean distance and reclassification
    euclidean_reclassify(feature_class, remap_table)

#### Build and execute raster calculator expression to generate technical inclusion layer

In [ ]:
# Get a list of reclassified rasters in the scratch folder
reclassified_rasters = get_rasters(scratch_folder)
reclassified_rasters

In [ ]:
# Build the raster calculator expression to calculate the final output raster
expression = "Con({condition}, 1)".format(
    condition=" & ".join([f'arcpy.sa.EqualTo("{raster}", 1)' for raster in reclassified_rasters])
)
expression

In [14]:
# Set the output path for the final raster
inclusion_raster = os.path.join(results_folder, "technicalInclusionArea_Wind.tif")
inclusion_raster_binary = os.path.join(results_folder, "technicalInclusionArea_Wind_binary.tif")

In [ ]:
# Execute raster calculator expression
arcpy.env.workspace = scratch_folder
arcpy.gp.RasterCalculator_sa(expression, inclusion_raster)

# Convert output raster to single bit depth
convert_to_one_bit(inclusion_raster, inclusion_raster_binary)

In [ ]:
# Delete scratch workspace
try:
    arcpy.Delete_management(scratch_folder)
except Exception as e:
         print(f"An error occurred while cleaning up: {str(e)}")